# Imports

In [197]:
# import stuff
import os, sys, time, json, csv, math, difflib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile as tif
from scipy.ndimage.interpolation import zoom

In [198]:
import warnings
warnings.simplefilter('ignore')

# User Inputs

In [293]:
# load label csvs 
labels_dir = '/Users/emilydennis/Desktop/lightsheet/labels_updated/'
lab_sigma = pd.read_csv(os.path.join(labels_dir,'SIGMA_labels_old.csv'))
lab_whs = pd.read_csv(os.path.join(labels_dir,'WHS_labels.csv'))
lab_schwarz = pd.read_csv(os.path.join(labels_dir,'schwarz_rl_annotations.csv'))
#lab_pma
lab_chon = pd.read_csv(os.path.join(labels_dir,'chon_full.csv'))

# load helper files
syns = pd.read_csv(os.path.join(labels_dir,'syns_with_parents.csv'))
PRAlabs = pd.read_csv(os.path.join(labels_dir,'PRA_labels_with-graph-order_children_parents.csv')).drop(columns=['Unnamed: 0'])

# load json
with open(os.path.join(labels_dir,'PRA_ontology.json')) as f:
  ontology = json.load(f)

In [211]:
syns

,common_name,name,parent
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
3076,raphe_nuclei,raphe,NaN
3077,thalamus_lateral_group,thalamus_dorsolateral,NaN
3078,thalamus_dorsal_nucleus,thalamus_midlinedorsal,NaN
3079,thalaus_ventromedial_group,thalamus_ventromedial,NaN


# Define Functions

In [200]:
def reformat(str_in):
    if isinstance(str_in,str):
        if str_in[-1]== ' ':
            str_in = str_in[:-1]
        str_in = str_in.replace(" ","_")
        str_in = str_in.replace("/","_")
        str_in = str_in.replace(",","")
        str_out = str_in.lower()
        
        str_out = str_out.replace("_left","")
        str_out = str_out.replace("_right","")
        str_out = str_out.replace("left","")
        str_out = str_out.replace("right","")
    else:
        print('{} is not a string'.format(str_in))
    return str_out

In [201]:
def replace_with_common_name(pd_in,syns):
    pd_out = pd_in.copy()
    for i in np.arange(0,len(pd_out.name)):
        lab = pd_out.name[i]
        if isinstance(lab,str):
            lab = reformat(lab)
            if (syns.name == lab).any(): 
                pd_out.name[i]=check_for_syn(lab,syns)
    return pd_out

In [202]:
def check_for_syn(name,syns):
    if name in list(syns.name):
        synonymns = syns[syns.name==name]['common_name']
    else: 
        synonymns=[]
    return synonymns

In [203]:
# get the parent name of a given name/volume
def get_parent_id_from_praid(labeledvolume,praid):
    parentid=list(labeledvolume.loc[labeledvolume.id == praid,'parent_id'])[0]
    return parentid

In [204]:
# get the hierarchy/family tree of a given name in a given volume
def get_hierarchy(labeledvolume,praid,maxlen):
    familytree = []
    parentid = get_parent_id_from_praid(labeledvolume,praid)
    # if parent isn't root, look for grandparent
    while parentid > 0:
        # add parent/grandparent... to tree
        familytree.append(parentid)
        # get grandparent/great grandparent...
        parentid=get_parent_id_from_praid(labeledvolume,parentid)
    familytree.reverse()
    print('family tree was {}'.format(familytree))
    if len(familytree) < 1:
        familytree=0
    return familytree   

In [205]:
# get all dissimilarity values for each overlapping set in two labeled volumes
def get_dissimilarity(Alab,Aval,Blab,Bval):
    nameA = get_name(Alab,Aval)
    print(nameA)
    familytreeA = get_hierarchy(Alab,nameA)
    nameB = get_name(Blab,Bval)
    print(nameB)
    familytreeB = get_hierarchy(Blab,nameB)
    if len(familytreeA) > len(familytreeB):
        # if family tree A longer than B
        diss = get_diss_value(familytreeA,familytreeB)
    elif len(familytreeB) > len(familytreeA):
        diss = get_diss_value(familytreeB,familytreeA)
    else:
        da = get_diss_value(familytreeA,familytreeB)
        db = get_diss_value(familytreeB,familytreeA)
        diss = min(da,db)
    return diss


# if you need to make a PRAlabs-like dataframe from a json file, extracting the group levels

In [16]:
# get children, parents for json entries
data = pd.json_normalize(data=ontology['children'])
data.insert(5,'has_children',0)
for i in data.index:
    print('i is {}'.format(i))
    if len(data.children[i])>0:
        data.has_children[i]=1
        contents=[]
        for j in np.arange(0,len(data.children[i])):
            print('j is {}'.format(j))
            contents.append(data.children[i][j]['id'])
        data.at[i,'children']=contents
new = data
val = 0

while len(new) > 0:
    val+=1
    new=pd.json_normalize(data=ontology['children'],record_path=np.repeat('children',val).tolist())
    if len(new)>0:
        new.insert(5,'has_children',0)
        for i in new.index:
            if len(new.children[i])>0:
                new.has_children[i]=1
                contents=[]
                for j in np.arange(0,len(new.children[i])):
                    contents.append(new.children[i][j]['id'])
                print(i)
                new.at[i,'children']=contents
    data = pd.concat([data,new])   
data=data.reset_index()

data.insert(6,'parent_ids',0)
for i in np.arange(0,max(data.graph_order)+1):
    go = max(data.graph_order)-i
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~ go is {}'.format(go))
    for j in data.index[data.graph_order==go]:
        curr_children = data.children[j]
        for k in data.children[j]:
            if len(data.children[data.id==k]) > 0: 
                curr_children = curr_children + list(data.children[data.id==k])[0]
        data.at[j,'children']=curr_children
        print('j {} ends as {}'.format(j,curr_children))

i is 0
j is 0
j is 1
j is 2
j is 3
j is 4
i is 1
j is 0
j is 1
j is 2
j is 3
j is 4
j is 5
j is 6
j is 7
j is 8
j is 9
j is 10
j is 11
j is 12
j is 13
j is 14
j is 15
j is 16
j is 17
j is 18
j is 19
i is 2
j is 0
j is 1
j is 2
j is 3
j is 4
j is 5
j is 6
j is 7
j is 8
j is 9
j is 10
j is 11
j is 12
j is 13
j is 14
i is 3
i is 4
j is 0
j is 1
j is 2
j is 3
i is 5
j is 0
j is 1
i is 6
i is 7
j is 0
i is 8
j is 0
j is 1
j is 2
j is 3
j is 4
j is 5
i is 9
i is 10
i is 11
i is 12
j is 0
j is 1
j is 2
j is 3
j is 4
j is 5
j is 6
j is 7
j is 8
j is 9
i is 13
i is 14
0
1
2
3
4
6
7
20
21
22
23
24
25
29
34
35
36
37
38
39
44
45
46
51
56
57
61
62
0
1
2
3
6
7
8
9
10
16
32
38
39
46
47
48
50
51
52
53
54
55
56
57
65
66
67
68
79
83
86
92
93
97
98
99
101
103
104
105
109
3
4
5
6
10
11
12
13
14
22
40
41
42
43
80
98
99
109
110
118
148
149
152
153
154
168
170
172
196
197
219
228
233
27

<ipython-input-16-38c5a67fb8d9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.has_children[i]=1
<ipython-input-16-38c5a67fb8d9>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new.has_children[i]=1



29
32
46
60
61
71
81
88
121
140
143
144
145
~~~~~~~~~~~~~~~~~~~~~~~~~~ go is 6
581
starts as []
ends as []
582
starts as []
ends as []
583
starts as []
ends as []
584
starts as []
ends as []
585
starts as []
ends as []
586
starts as []
ends as []
587
starts as []
ends as []
588
starts as []
ends as []
589
starts as []
ends as []
590
starts as []
ends as []
591
starts as []
ends as []
592
starts as []
ends as []
593
starts as []
ends as []
594
starts as []
ends as []
595
starts as []
ends as []
596
starts as []
ends as []
597
starts as []
ends as []
598
starts as []
ends as []
599
starts as []
ends as []
600
starts as []
ends as []
601
starts as []
ends as []
602
starts as []
ends as []
603
starts as []
ends as []
604
starts as []
ends as []
605
starts as []
ends as []
606
starts as []
ends as []
607
starts as []
ends as []
608
starts as []
ends as []
609
starts as []
ends as []
610
starts as []
ends as []
611
starts as []
ends as []
612
starts as []
ends as []
613
starts as []
ends as

ends as [367, 368, 369, 370, 371, 372]
386
starts as [469, 470, 471, 472, 473, 606, 607]
ends as [469, 470, 471, 472, 473, 606, 607, 457, 458, 459, 460, 461, 462, 474, 357, 475, 476, 477]
387
starts as []
ends as []
388
starts as []
ends as []
389
starts as []
ends as []
390
starts as []
ends as []
391
starts as []
ends as []
392
starts as []
ends as []
393
starts as []
ends as []
394
starts as []
ends as []
395
starts as []
ends as []
396
starts as []
ends as []
397
starts as []
ends as []
398
starts as []
ends as []
399
starts as []
ends as []
400
starts as []
ends as []
401
starts as []
ends as []
402
starts as []
ends as []
403
starts as []
ends as []
404
starts as []
ends as []
405
starts as []
ends as []
406
starts as []
ends as []
407
starts as []
ends as []
408
starts as [74, 75]
ends as [74, 75, 132, 133]
409
starts as []
ends as []
410
starts as []
ends as []
411
starts as []
ends as []
412
starts as []
ends as []
413
starts as []
ends as []
414
starts as []
ends as []
415
st

ends as [38, 106, 107, 108, 109, 110, 111, 575, 576, 605, 137, 138, 99, 100, 101, 102, 103, 478, 479, 480, 481]
50
starts as [62]
ends as [62]
51
starts as [68, 69, 70, 71, 72, 73]
ends as [68, 69, 70, 71, 72, 73]
52
starts as [87, 88, 89, 574]
ends as [87, 88, 89, 574, 90]
53
starts as [181, 182, 183]
ends as [181, 182, 183]
54
starts as [588]
ends as [588, 259, 260, 261]
55
starts as []
ends as []
56
starts as []
ends as []
57
starts as []
ends as []
58
starts as []
ends as []
59
starts as [85, 86]
ends as [85, 86]
60
starts as [549, 550]
ends as [549, 550, 78, 79, 80, 81, 82, 83]
61
starts as [499, 500]
ends as [499, 500]
62
starts as []
ends as []
63
starts as []
ends as []
64
starts as []
ends as []
65
starts as []
ends as []
66
starts as [253, 254]
ends as [253, 254]
67
starts as []
ends as []
68
starts as []
ends as []
69
starts as []
ends as []
70
starts as []
ends as []
71
starts as [116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 596]
ends as [116, 117, 118, 119, 120, 

# Example of how to take a labeled volume and relate to PRA

In [294]:
lab_atl = lab_chon.copy()

if 'name' in lab_atl.columns:
    lab_atl.columns=['label', 'ID']
lab_atl

,atlas,label,abbr,ID,structureOrder,ParentID,Parentabbreviation,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,allen,Basic cell groups and regions,grey,8,100,997,root,NaN,NaN,NaN,NaN,NaN
1,allen,Cerebrum,CH,567,200,8,grey,NaN,NaN,NaN,NaN,NaN
2,allen,Cerebral cortex,CTX,688,300,567,CH,NaN,NaN,NaN,NaN,NaN
3,allen,Cortical plate,CTXpl,695,400,688,CTX,NaN,NaN,NaN,NaN,NaN
4,allen,Isocortex,Isocortex,315,500,695,CTXpl,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2385,fp,posterolateral fissure,plf,11,118300,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN
2386,fp,intercrural fissure,icf,34,118600,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN
2387,fp,ansoparamedian fissure,apmf,43,118700,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN
2388,fp,parafloccular sulcus,pfs,65,119000,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN


In [295]:
lab_atl.insert(2,'common_name','null')
lab_atl.insert(2,'name','null')
lab_atl

,atlas,label,name,common_name,abbr,ID,structureOrder,ParentID,Parentabbreviation,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,allen,Basic cell groups and regions,null,null,grey,8,100,997,root,NaN,NaN,NaN,NaN,NaN
1,allen,Cerebrum,null,null,CH,567,200,8,grey,NaN,NaN,NaN,NaN,NaN
2,allen,Cerebral cortex,null,null,CTX,688,300,567,CH,NaN,NaN,NaN,NaN,NaN
3,allen,Cortical plate,null,null,CTXpl,695,400,688,CTX,NaN,NaN,NaN,NaN,NaN
4,allen,Isocortex,null,null,Isocortex,315,500,695,CTXpl,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2385,fp,posterolateral fissure,null,null,plf,11,118300,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN
2386,fp,intercrural fissure,null,null,icf,34,118600,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN
2387,fp,ansoparamedian fissure,null,null,apmf,43,118700,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN
2388,fp,parafloccular sulcus,null,null,pfs,65,119000,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN


In [296]:
manual_check=[]
for i in lab_atl.index:
    print(i)
    lab_atl.at[i,'name']=reformat(lab_atl.label[i])
    nm=lab_atl.at[i,'name']
    if nm in list(syns.common_name):
        print('if')
        lab_atl.at[i,'common_name']=nm
    elif nm in list(syns.name):
        print('elif')
        lab_atl.at[i,'common_name']=list(syns.common_name[syns.name==nm])[0]
    else:
        manual_check.append(i)

0
if
1
if
2
elif
3
elif
4
elif
5
elif
6
elif
7
elif
8
elif
9
elif
10
elif
11
elif
12
elif
13
elif
14
elif
15
elif
16
elif
17
elif
18
elif
19
elif
20
elif
21
elif
22
elif
23
elif
24
elif
25
elif
26
elif
27
elif
28
elif
29
elif
30
elif
31
elif
32
elif
33
34
elif
35
elif
36
elif
37
elif
38
elif
39
elif
40
elif
41
elif
42
elif
43
elif
44
elif
45
elif
46
elif
47
elif
48
elif
49
elif
50
elif
51
elif
52
elif
53
elif
54
elif
55
elif
56
elif
57
elif
58
elif
59
elif
60
elif
61
elif
62
elif
63
elif
64
elif
65
elif
66
elif
67
elif
68
elif
69
elif
70
elif
71
elif
72
elif
73
elif
74
elif
75
elif
76
elif
77
elif
78
elif
79
elif
80
elif
81
elif
82
elif
83
elif
84
elif
85
elif
86
elif
87
elif
88
elif
89
elif
90
elif
91
elif
92
elif
93
elif
94
elif
95
elif
96
elif
97
elif
98
elif
99
elif
100
elif
101
elif
102
elif
103
elif
104
elif
105
elif
106
elif
107
elif
108
elif
109
elif
110
elif
111
elif
112
elif
113
elif
114
elif
115
elif
116
elif
117
elif
118
elif
119
elif
120
elif
121
elif
122
elif
123
elif
124

elif
1010
elif
1011
elif
1012
elif
1013
elif
1014
elif
1015
elif
1016
elif
1017
elif
1018
elif
1019
elif
1020
elif
1021
elif
1022
elif
1023
elif
1024
elif
1025
elif
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
elif
1040
1041
1042
1043
1044
1045
1046
elif
1047
elif
1048
elif
1049
elif
1050
elif
1051
elif
1052
elif
1053
elif
1054
elif
1055
elif
1056
elif
1057
elif
1058
elif
1059
elif
1060
if
1061
elif
1062
elif
1063
elif
1064
elif
1065
elif
1066
elif
1067
elif
1068
elif
1069
elif
1070
elif
1071
elif
1072
elif
1073
elif
1074
elif
1075
elif
1076
elif
1077
elif
1078
elif
1079
elif
1080
elif
1081
elif
1082
elif
1083
elif
1084
elif
1085
elif
1086
elif
1087
elif
1088
if
1089
elif
1090
elif
1091
elif
1092
elif
1093
elif
1094
elif
1095
elif
1096
if
1097
elif
1098
elif
1099
elif
1100
elif
1101
elif
1102
elif
1103
elif
1104
elif
1105
elif
1106
elif
1107
elif
1108
elif
1109
elif
1110
if
1111
elif
1112
elif
1113
elif
1114
elif
1115
elif
1116
elif
1117
elif
1118
elif
1119
eli

elif
2052
elif
2053
elif
2054
elif
2055
2056
2057
2058
2059
elif
2060
elif
2061
elif
2062
elif
2063
if
2064
elif
2065
elif
2066
elif
2067
elif
2068
elif
2069
2070
2071
elif
2072
if
2073
elif
2074
elif
2075
elif
2076
elif
2077
if
2078
elif
2079
2080
2081
if
2082
if
2083
elif
2084
elif
2085
2086
elif
2087
elif
2088
elif
2089
elif
2090
2091
elif
2092
elif
2093
elif
2094
if
2095
2096
2097
2098
elif
2099
if
2100
elif
2101
elif
2102
elif
2103
2104
2105
2106
2107
2108
2109
2110
if
2111
if
2112
if
2113
2114
if
2115
elif
2116
elif
2117
elif
2118
elif
2119
2120
2121
elif
2122
elif
2123
elif
2124
2125
if
2126
if
2127
2128
2129
if
2130
elif
2131
elif
2132
elif
2133
if
2134
elif
2135
elif
2136
elif
2137
2138
elif
2139
elif
2140
2141
2142
2143
2144
2145
2146
elif
2147
elif
2148
elif
2149
elif
2150
elif
2151
if
2152
elif
2153
elif
2154
elif
2155
elif
2156
2157
2158
2159
if
2160
2161
elif
2162
elif
2163
2164
elif
2165
elif
2166
elif
2167
elif
2168
elif
2169
elif
2170
elif
2171
elif
2172
if
2173
elif
2

In [297]:
manual_check

[33,
 706,
 707,
 709,
 710,
 711,
 750,
 751,
 752,
 757,
 820,
 821,
 822,
 874,
 970,
 978,
 979,
 980,
 981,
 982,
 997,
 998,
 999,
 1000,
 1001,
 1002,
 1003,
 1008,
 1026,
 1027,
 1028,
 1029,
 1030,
 1031,
 1032,
 1033,
 1034,
 1035,
 1036,
 1037,
 1038,
 1040,
 1041,
 1042,
 1043,
 1044,
 1045,
 1135,
 1136,
 1137,
 1138,
 1139,
 1140,
 1163,
 1164,
 1165,
 1166,
 1167,
 1168,
 1170,
 1171,
 1172,
 1173,
 1174,
 1175,
 1177,
 1216,
 1217,
 1218,
 1219,
 1220,
 1232,
 1233,
 1234,
 1235,
 1236,
 1279,
 1280,
 1281,
 1282,
 1283,
 1284,
 1286,
 1287,
 1288,
 1289,
 1290,
 1291,
 1293,
 1294,
 1295,
 1296,
 1297,
 1298,
 1310,
 1311,
 1312,
 1313,
 1314,
 1315,
 1336,
 1363,
 1368,
 1369,
 1370,
 1371,
 1400,
 1443,
 1456,
 1457,
 1458,
 1459,
 1460,
 1495,
 1496,
 1500,
 1501,
 1503,
 1504,
 1505,
 1521,
 1522,
 1523,
 1524,
 1525,
 1526,
 1527,
 1528,
 1529,
 1530,
 1531,
 1532,
 1533,
 1534,
 1535,
 1536,
 1537,
 1538,
 1539,
 1540,
 1541,
 1542,
 1543,
 1544,
 1545,
 1546,
 1

In [301]:
# for each element of manual_check, we print the name, and closest options
# add to syns csv, and re-import!
idval=manual_check[0]
print(list(lab_atl[lab_atl.index==idval]['name'])[0])
print(difflib.get_close_matches(list(lab_atl[lab_atl.index==idval]['name'])[0],list(PRAlabs.name),n=20,cutoff=0.1))

nucleus_raphè_magnus
['linear_nucleus_raphe_caudal', 'nucleus_of_the_lateral_lemniscus', 'nucleus_y', 'nucleus_x', 'raphe_magnus_nucleus', 'septal_nucleus_triangular', 'raphe_nucleus_medial', 'raphe_nucleus_dorsal', 'pontine_nuclei_raphe', 'superior_central_nucleus_raphe', 'terminal_nucleus_dorsal', 'globus_pallidus', 'cochlear_nucleus_ventral', 'red_nucleus', 'corpus_callosum_genu', 'tegmetal_nucleus_anterior', 'tegmental_nucleus_ventral', 'barrington_nucleus_dorsal', 'midbrain_raphe_nuclei', 'pretectal_nucleus_anterior']


In [302]:
# great! now we have common_names that match our PRA ID info. 
# Merge that info into your volume labels.
lab_atl=lab_atl.assign(parent_id=np.nan, graph_order=0.0, children='null', has_children=0,parent_ids='null')

In [303]:
lab_atl #make sure everything makes sense and looks reasonable

,atlas,label,name,common_name,abbr,ID,structureOrder,ParentID,Parentabbreviation,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,parent_id,graph_order,children,has_children,parent_ids
0,allen,Basic cell groups and regions,basic_cell_groups_and_regions,basic_cell_groups_and_regions,grey,8,100,997,root,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
1,allen,Cerebrum,cerebrum,cerebrum,CH,567,200,8,grey,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
2,allen,Cerebral cortex,cerebral_cortex,cortex,CTX,688,300,567,CH,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
3,allen,Cortical plate,cortical_plate,cortex,CTXpl,695,400,688,CTX,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
4,allen,Isocortex,isocortex,cortex,Isocortex,315,500,695,CTXpl,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2385,fp,posterolateral fissure,posterolateral_fissure,posterolateral_fissure,plf,11,118300,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
2386,fp,intercrural fissure,intercrural_fissure,fissure_intercrural,icf,34,118600,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
2387,fp,ansoparamedian fissure,ansoparamedian_fissure,ansoparamedian_fissure,apmf,43,118700,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
2388,fp,parafloccular sulcus,parafloccular_sulcus,paraflocculus_sulcus,pfs,65,119000,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null


In [218]:
PRAlabs

,id,name,parent_id,graph_order,children,has_children,parent_ids
0,35,brainstem,0,1,"[39, 40, 41, 577, 595, 189, 190, 265, 266, 267...",1,0
1,36,cerebellum,0,1,"[43, 51, 52, 53, 54, 55, 127, 128, 129, 184, 1...",1,0
2,37,cerebrum,0,1,"[59, 63, 64, 65, 66, 67, 398, 564, 565, 567, 5...",1,0
3,243,corpus_collosum,0,1,[],0,0
4,244,corticospinal_tract,0,1,"[112, 113, 114, 115]",1,0
...,...,...,...,...,...,...,...
612,475,accumbens_nucleus,607,6,[],0,"[37, 66, 34, 61, 607]"
613,476,olfactory_tubercle,607,6,[],0,"[37, 66, 34, 61, 607]"
614,477,striatum_fundus,607,6,[],0,"[37, 66, 34, 61, 607]"
615,132,cuneate_fascicle,74,6,[],0,"[450, 157, 116, 136, 74]"


In [246]:
manual_check=[]
for i in lab_atl.index:
    print(i)
    atl_nm = lab_atl.common_name[i]
    df = PRAlabs[PRAlabs.name==lab_atl.common_name[i]].reset_index()
    if len(df)==0:
        manual_check.append(i)
    else:
        lab_atl.parent_id[i]=df.parent_id.values[0]
        lab_atl.graph_order[i]=df.graph_order
        lab_atl.children[i]=df.at[0,'children']
        lab_atl.has_children[i]=df.has_children
        lab_atl.parent_ids=df.at[0,'parent_ids']


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [247]:
manual_check#if you updated syns but there are still issues...
# make sure you re-ran user inputs cell, to do so, uncomment below
# then re-run first manual_check loop
#syns = pd.read_csv(os.path.join(labels_dir,'syns_with_parents.csv'))


[]

In [248]:
lab_atl.to_csv(os.path.join(labels_dir,'schwarz_withPRA.csv'))

,ID,Franklin-Paxinos abbreviation,name,common_name,Franklin-Paxinos Full name,Unnamed: 3,struct_order,red,green,blue,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,1,VTM,null,null,Ventral tuberomammillary nucleus,NaN,75500.0,255.0,78.0,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,S1ULp-6b,null,null,"Primary somatosensory cortex, upper lip region...",NaN,6200.0,137.0,205.0,161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,sf,null,null,secondary fissure,NaN,118200.0,170.0,170.0,170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,IC,null,null,Inferior colliculus,NaN,79100.0,255.0,152.0,255.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,ic,null,null,internal capsule,NaN,107000.0,204.0,204.0,204.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4284,58500,8Cb-mo,null,null,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4285,59100,9Cb-gr,null,null,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4286,58900,9Cb-mo,null,null,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4287,59500,10Cb-gr,null,null,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [312]:
lab_atl = lab_atl.rename(columns={"structureOrder":'struct_order'})

In [305]:
PRAlabs.insert(4,'struct_order',0)

In [306]:
PRAlabs

,id,name,parent_id,graph_order,struct_order,children,has_children,parent_ids
0,35,brainstem,0,1,0,"[39, 40, 41, 577, 595, 189, 190, 265, 266, 267...",1,0
1,36,cerebellum,0,1,0,"[43, 51, 52, 53, 54, 55, 127, 128, 129, 184, 1...",1,0
2,37,cerebrum,0,1,0,"[59, 63, 64, 65, 66, 67, 398, 564, 565, 567, 5...",1,0
3,243,corpus_collosum,0,1,0,[],0,0
4,244,corticospinal_tract,0,1,0,"[112, 113, 114, 115]",1,0
...,...,...,...,...,...,...,...,...
612,475,accumbens_nucleus,607,6,0,[],0,"[37, 66, 34, 61, 607]"
613,476,olfactory_tubercle,607,6,0,[],0,"[37, 66, 34, 61, 607]"
614,477,striatum_fundus,607,6,0,[],0,"[37, 66, 34, 61, 607]"
615,132,cuneate_fascicle,74,6,0,[],0,"[450, 157, 116, 136, 74]"


In [307]:
lab_atl

,atlas,label,name,common_name,abbr,ID,structureOrder,ParentID,Parentabbreviation,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,parent_id,graph_order,children,has_children,parent_ids
0,allen,Basic cell groups and regions,basic_cell_groups_and_regions,basic_cell_groups_and_regions,grey,8,100,997,root,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
1,allen,Cerebrum,cerebrum,cerebrum,CH,567,200,8,grey,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
2,allen,Cerebral cortex,cerebral_cortex,cortex,CTX,688,300,567,CH,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
3,allen,Cortical plate,cortical_plate,cortex,CTXpl,695,400,688,CTX,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
4,allen,Isocortex,isocortex,cortex,Isocortex,315,500,695,CTXpl,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2385,fp,posterolateral fissure,posterolateral_fissure,posterolateral_fissure,plf,11,118300,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
2386,fp,intercrural fissure,intercrural_fissure,fissure_intercrural,icf,34,118600,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
2387,fp,ansoparamedian fissure,ansoparamedian_fissure,ansoparamedian_fissure,apmf,43,118700,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null
2388,fp,parafloccular sulcus,parafloccular_sulcus,paraflocculus_sulcus,pfs,65,119000,1040,grv_of_CBX,NaN,NaN,NaN,NaN,NaN,NaN,0.0,null,0,null


In [315]:
manual_check=[]
for i in PRAlabs.index:
    print(i)
    atl_nm = PRAlabs.name[i]
    df = lab_atl[lab_atl.common_name==PRAlabs.name[i]].reset_index()
    if len(df)==0:
        manual_check.append(i)
    else:
        PRAlabs.struct_order[i] = df.struct_order[0]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [317]:
PRAlabs.to_csv('/Users/emilydennis/Desktop/lightsheet/labels_updated/pra_with_chon_structorder.csv')